# Homework 1: Due February 13 at 3:00PM.
Fork/clone this repo and then solve the problems in this notebook. Make sure your code is well-commented, separated into different blocks in a smart way, and executed before your push it to **your** repository.

## Problem 1: Coding, integration, and a simple econ example
A profit-maximizing firm faces a demand curve given by: $P(q) = a-bq$ where $b\sim logN(\mu,\sigma)$ and has a cost function given by $C(q) = cq$. 

1. Solve for the optimal quantity analytically. Write your proof in the notebook using markdown.
2. Write a function called `profit_max_q(a, c, mu, sigma, method, n)` that returns the numerical optimal quantity given a set of inputs $(a, c, \mu, \sigma, method, n)$, where `method` is a string that takes on a value of `"mc"` or `"quad"` and determines whether you integrate using Monte Carlo or quadrature methods, and `n` is the number of Monte Carlo draws or quadrature nodes.
3. Choose a set of values $(a, c, \mu, \sigma)$ and use `profit_max_q` to solve the problem for both approaches to integration. Use the `CompEcon` package to implement the quadrature routine.

## Problem 1 Answer
1.1 
We want to solve $\max_{q}\mathbb{E}(\pi(q)) = \max_{q}\mathbb{E}(aq - bq^{2} -cq)$.
Taking FOC gives $a - 2q\mathbb{E}(b) - c = 0$.  Rearranging gives \begin{equation}
q = \frac{a-c}{2\mathbb{E}(b)} = \frac{a-c}{2exp(\mu+\frac{\sigma^{2}}{2})}
\end{equation}
because $b\sim logN(\mu,\sigma)$.

In [6]:
#Pkg.add("CompEcon")
using CompEcon
#Pkg.add("FastGaussQuadrature")
using FastGaussQuadrature


function lognpdf(mu, sigma, x)
    #evaluates logN(mu,sigma) pdf at x
    if x<0
        println("x must be positive")
    else
    return (x*sigma*sqrt(2*pi))^(-1)*exp(-(log(x)-mu)^(2)/(2*sigma^(2)))
    end
end


function profit_max_q(a, c, mu, sigma, method, n)
    #this function solves for q that maximizes profits given by pi=aq-bq^2-cq  where b~logN(mu,sigma)
    if method=="quad"
        nodes, weights = gausslegendre(n)
        Eb = sum(weights.*(exp.(nodes).*lognpdf.(mu,sigma,exp.(nodes))))
        q = (a-c)/2*Eb
    elseif method=="mc"
        Random.seed!(1234)
        nodes = rand(n)
        Eb = sum(exp.(nodes).*lognpdf.(mu,sigma,exp.(nodes)))
        q = (a-c)/2*Eb
    else
        println("Method must be either "mc" or "quad"")
    end
    return q
end

q_quad = profit_max_q(3, 1, .5, 1, "quad", 5)

println("Using guassian quadrature, we estimate optimal quantity = $q_quad")

q_mc = profit_max_q(3, 1, .5, 1, "mc", 100)

println("Using MC, we estimate optimal quantity = $q_mc")

1.2


LoadError: syntax: cannot juxtapose string literal

## Problem 2: Coding and Monte Carlo
Approximate $\pi$ using Monte Carlo. You may only use `rand()` to generate random numbers. Here is how to think about approximating $\pi$:
1. Suppose $U$ is a two dimensional random variable on the unit square $[0,1]\times[0,1]$. The probability that $U$ is in a subset $B$ of $(0,1)\times(0,1)$ is equal to the area of $B$.
2. If $u_1,...,u_n$ are iid draws from $U$, then as $n$ grows (by an LLN type argument), the fraction that falls inside $B$ is the probability of another iid draw coming from $B$.
3. The area of a circle is given by $\pi \times radius^2$.

## Problem 2 Answer

In [16]:
using Random

Random.seed!(1234)

function dev2(x)
    (x-0.5)^(2)
end

Size =  [1000,10000,100000,1000000] 

for n in Size
    U = rand(n,2)
    U1 = dev2.(U)
    U2 = sum(U1,1)
    dist = sqrt.(U2)
    incircle = length(dist[dist .< 0.5])
    Area = incircle/n
    pi_est = Area*(.5)^(-2)
    
    println("With $n random draws, we estimate pi to be $pi_est.")
end
    

MethodError: MethodError: objects of type Array{Float64,2} are not callable
Use square brackets [] for indexing an Array.

## Problem 3: Git

1. Create a new repository named `problem-set-1-q-3` on your GitHub account.
2. Put in a `README.md` with the following text: `Hello World!`.
3. Put in a .gitignore file, ignoring the Jupyter files .ipynb_checkpoints and the project files, .projects.
4. Create a new branch called `new-branch`.
5. Change the `README.md` text to `Goodbye World!`.
6. Merge `new-branch` back into `master`.

## Problem 3 Answer

See https://github.com/m-ingram/problem-set-1-q-3

## Problem 4: Memory location

Let's learn about some of the nuances of memory allocation.

1. Generate one $20000 \times 20000$ array of random numbers named `x`. 
2. Make a function called `exp_cols` which exponentiates the elements of `x` column by column (i.e. by broadcasting `exp.()`) and returns the exponentiated array.
3. Make a function called `exp_rows` which exponentiates the elements of `x` row by row (i.e. by broadcasting `exp.()`) and returns the exponentiated array.
4. Call `exp_cols(x)` and `exp_rows(x)` twice and calculate the elapsed time on the second call (avoids fixed cost of initial compiliation).
5. Is one faster than the other?

In [1]:
using Random

Random.seed!(1234)

x = rand(20000,20000)

#Computing exp.(x) causes my jupyter notebook kernal to die so I've done the problem with summing the rows/columns

function sum_cols(x)
    return sum(x,2)
end

function sum_rows(x)
    return sum(x,1)
end

sum_cols(x)
@time sum_cols(x)

sum_rows(x)
@time sum_rows(x)


MethodError: MethodError: objects of type Array{Float64,2} are not callable
Use square brackets [] for indexing an Array.

## Comment on code

The julia code does not run and I have been able to resolve the errors yet. It's not clear to me why arrays are not callable into functions. The Julia language doc references an array not being callable in a case such as x(x) instead of specifying multiplication.  But that isn't happening here.   I'm going to try to redo this in python.